In [65]:
import numpy as np
import pandas as pd
import re
import spacy
from gensim.models import Word2Vec
import gensim.downloader as api
from gensim.models import KeyedVectors
import warnings
warnings.filterwarnings("ignore", message="NotOpenSSLWarning")
# !pip install transformers
# !pip install minicons
from minicons import scorer
import torch
import csv

In [66]:
spacy_transcripts = pd.read_csv('processed_data/transcripts_spacy_formatted.csv')
spacy_linguistic_features = spacy_transcripts[['Coherence']].copy()
display(spacy_linguistic_features)

,Coherence
0,3
1,4
2,4
3,4
4,5
...,...
1534,3
1535,4
1536,3
1537,5


In [67]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Part-of-speech proportions, open/closed-class word ratio, and propositional density

# TODO: make this into a function?

pos_tags = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SPACE', 'SYM', 'VERB', 'X']
open_class_tags = ['NOUN', 'VERB', 'ADJ', 'ADV']
closed_class_tags = ['ADP', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NUM', 'PART', 'PRON', 'SCONJ']
propositional_density_tags = ['ADJ', 'ADV', 'ADP', 'CCONJ', 'SCONJ', 'VERB']

for i in spacy_linguistic_features.index:  

    string = spacy_transcripts['Text'].iloc[i]
    doc = nlp(string)
    testing = []
    open_class = 0
    closed_class = 0
    prop_density = 0

    for token in doc:
        testing.append(token.pos_)
        if token.pos_ in open_class_tags:
            open_class += 1
        elif token.pos_ in closed_class_tags:
            closed_class += 1
        if token.pos_ in propositional_density_tags:
            prop_density += 1
    
    for count_index, pos in enumerate(pos_tags): # TODO: try...except for division by 0
        spacy_linguistic_features.loc[i, f'POS:{pos}'] = testing.count(pos)/len(testing) # TODO: make sure this covers everything, and add checks
   
    try:
        spacy_linguistic_features.loc[i, 'open_closed_ratio'] = open_class / closed_class
    except ZeroDivisionError:
        print(f"Index {i} has no closed class words")

    spacy_linguistic_features.loc[i, 'propositional_density'] = prop_density / len(testing)

display(spacy_linguistic_features)

In [ ]:
# TODO - make this better and less confusing - import story lists from the other thing

story_lists = []
with open('processed_data/sentences.csv', 'r', newline='') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        story_lists.append(row)

In [ ]:
# Number of utterances

for i, story in enumerate(story_lists):
    spacy_linguistic_features.loc[i, 'n_utterances'] = len(story)

,Coherence,POS:ADJ,POS:ADP,POS:ADV,POS:AUX,POS:CCONJ,POS:DET,POS:INTJ,POS:NOUN,POS:NUM,...,open_closed_ratio,propositional_density,n_logical_operators,log10_freq_mean,semantic_diversity_mean,aoa_mean,concreteness_mean,semantic_thematic_distance,surprisal,n_utterances
0,3,0.044534,0.064777,0.076923,0.068826,0.020243,0.093117,0.000000,0.141700,0.008097,...,0.820513,0.356275,4.0,4.682792,2.107979,4.616221,2.597958,0.198185,4.895232,21.0
1,4,0.043046,0.076159,0.076159,0.059603,0.026490,0.076159,0.000000,0.109272,0.009934,...,0.788732,0.380795,7.0,4.666692,2.069585,4.606250,2.615833,0.195398,4.183853,21.0
2,4,0.054054,0.091892,0.055135,0.060541,0.022703,0.077838,0.002162,0.139459,0.009730,...,0.879795,0.375135,17.0,4.396347,2.102881,5.347357,2.420347,0.180549,5.146196,82.0
3,4,0.071806,0.080253,0.069694,0.055966,0.030623,0.080253,0.001056,0.165787,0.013728,...,1.036082,0.376980,30.0,4.421817,2.076899,5.135905,2.673849,0.178148,5.596125,86.0
4,5,0.033333,0.038889,0.033333,0.061111,0.044444,0.094444,0.000000,0.177778,0.016667,...,0.878049,0.338889,9.0,4.596490,2.064892,5.063814,2.591143,0.161879,4.702588,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,3,0.084746,0.059322,0.072034,0.067797,0.055085,0.042373,0.000000,0.165254,0.004237,...,1.067961,0.453390,10.0,4.404882,2.118607,5.127013,2.415114,0.204966,3.565189,14.0
1535,4,0.074074,0.055556,0.129630,0.074074,0.046296,0.055556,0.000000,0.111111,0.000000,...,1.000000,0.481481,3.0,4.578900,2.106413,5.049481,2.350449,0.219363,3.443641,6.0
1536,3,0.071429,0.098214,0.044643,0.044643,0.035714,0.107143,0.000000,0.214286,0.008929,...,1.061224,0.419643,4.0,4.368046,2.086000,5.140270,2.389186,0.172722,4.055240,7.0
1537,5,0.032895,0.065789,0.052632,0.072368,0.032895,0.078947,0.000000,0.138158,0.013158,...,0.830986,0.368421,3.0,4.474590,2.176371,5.095055,2.484435,0.196328,3.690323,11.0


In [ ]:
# Number of words per utterance

for i, story in enumerate(story_lists):
    sentence_lengths = []
    for sentence in story:
        replaced_sentence = re.sub(r'[^A-Za-z0-9\s]', '', sentence)
        replaced_sentence = re.sub(r' +|\n', ' ', replaced_sentence)
        split_sentence = replaced_sentence.split(' ')
        split_sentence = [word for word in split_sentence if word != '']
        length = len(split_sentence)
        sentence_lengths.append(length)

    sentence_array = np.array(sentence_lengths)
    mean = np.nanmean(sentence_lengths)
    spacy_linguistic_features.loc[i, 'n_words_per_utterance'] = mean

In [ ]:
# Logical operators (and, not, or, if ... then)

logical_operators = [
    (r' and[!?\-,.:;\'"\s]'), # TODO: account for And at the beginning of a sentence (^And?)
    (r' not[!?\-,.:;\'"\s]'),
    (r' or[!?\-,.:;\'"\s]'),
    (r' (if) (.*?)(then)[!?\-…,.:;\'"\s]')]

for i in range(len(story_lists)):
    logical_operator_number = []
    current_list = story_lists[i]
    for j in range(len(current_list)):
        string = current_list[j]
        for pattern in logical_operators:
            match = re.findall(pattern, string, flags = re.IGNORECASE)
            n_operators = len(match)
            logical_operator_number.append(n_operators)
    spacy_linguistic_features.loc[i, 'n_logical_operators'] = np.sum(logical_operator_number)

display(spacy_linguistic_features)

,Coherence,POS:ADJ,POS:ADP,POS:ADV,POS:AUX,POS:CCONJ,POS:DET,POS:INTJ,POS:NOUN,POS:NUM,...,POS:X,open_closed_ratio,propositional_density,n_logical_operators,log10_freq_mean,semantic_diversity_mean,aoa_mean,concreteness_mean,semantic_thematic_distance,surprisal
0,3,0.044534,0.064777,0.076923,0.068826,0.020243,0.093117,0.000000,0.141700,0.008097,...,0.0,0.820513,0.356275,4.0,4.682792,2.107979,4.616221,2.597958,0.198185,4.895232
1,4,0.043046,0.076159,0.076159,0.059603,0.026490,0.076159,0.000000,0.109272,0.009934,...,0.0,0.788732,0.380795,7.0,4.666692,2.069585,4.606250,2.615833,0.195398,4.183853
2,4,0.054054,0.091892,0.055135,0.060541,0.022703,0.077838,0.002162,0.139459,0.009730,...,0.0,0.879795,0.375135,17.0,4.396347,2.102881,5.347357,2.420347,0.180549,5.146196
3,4,0.071806,0.080253,0.069694,0.055966,0.030623,0.080253,0.001056,0.165787,0.013728,...,0.0,1.036082,0.376980,30.0,4.421817,2.076899,5.135905,2.673849,0.178148,5.596125
4,5,0.033333,0.038889,0.033333,0.061111,0.044444,0.094444,0.000000,0.177778,0.016667,...,0.0,0.878049,0.338889,9.0,4.596490,2.064892,5.063814,2.591143,0.161879,4.702588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,3,0.084746,0.059322,0.072034,0.067797,0.055085,0.042373,0.000000,0.165254,0.004237,...,0.0,1.067961,0.453390,10.0,4.404882,2.118607,5.127013,2.415114,0.204966,3.565189
1535,4,0.074074,0.055556,0.129630,0.074074,0.046296,0.055556,0.000000,0.111111,0.000000,...,0.0,1.000000,0.481481,3.0,4.578900,2.106413,5.049481,2.350449,0.219363,3.443641
1536,3,0.071429,0.098214,0.044643,0.044643,0.035714,0.107143,0.000000,0.214286,0.008929,...,0.0,1.061224,0.419643,4.0,4.368046,2.086000,5.140270,2.389186,0.172722,4.055240
1537,5,0.032895,0.065789,0.052632,0.072368,0.032895,0.078947,0.000000,0.138158,0.013158,...,0.0,0.830986,0.368421,3.0,4.474590,2.176371,5.095055,2.484435,0.196328,3.690323


In [ ]:
# TODO: adapt this so it works better (put it in the preprocessing script, then make it so that it is called/read here)

coherence_processed_transcripts = spacy_transcripts.copy()
for i in spacy_transcripts.index:
    story_string = spacy_transcripts['Text'].iloc[i]
    story_string_replaced = re.sub(r'[^A-Za-z0-9\s]', '', story_string)
    story_string_replaced = re.sub(r' +|\n', ' ', story_string_replaced)
    coherence_processed_transcripts.loc[i,'Text'] = story_string_replaced

In [ ]:
# Number of words and type-token ratio

def string_ttr(string: str) -> float:
    # TODO: type hints?
    string_lower = string.lower()
    string_split = string_lower.split(' ')
    string_set = set(string_split)
    n_tokens = len(string_split)
    types = len(string_set)
    try:
        ttr = types / n_tokens
    except ZeroDivisionError:
        print(f"Input string has no tokens (empty string)")

    if ttr > 1:
        raise ValueError('Type-token ratio cannot exceed 1')

    return ttr, n_tokens

# TODO: make this formatting better

for i in coherence_processed_transcripts.index:  

    string = coherence_processed_transcripts['Text'].iloc[i]
    coherence_processed_transcripts.loc[i, 'type_token_ratio'], coherence_processed_transcripts.loc[i, 'n_words'] = string_ttr(string)

display(spacy_linguistic_features)

,Coherence,POS:ADJ,POS:ADP,POS:ADV,POS:AUX,POS:CCONJ,POS:DET,POS:INTJ,POS:NOUN,POS:NUM,...,open_closed_ratio,propositional_density,n_logical_operators,log10_freq_mean,semantic_diversity_mean,aoa_mean,concreteness_mean,semantic_thematic_distance,surprisal,n_utterances
0,3,0.044534,0.064777,0.076923,0.068826,0.020243,0.093117,0.000000,0.141700,0.008097,...,0.820513,0.356275,4.0,4.682792,2.107979,4.616221,2.597958,0.198185,4.895232,21.0
1,4,0.043046,0.076159,0.076159,0.059603,0.026490,0.076159,0.000000,0.109272,0.009934,...,0.788732,0.380795,7.0,4.666692,2.069585,4.606250,2.615833,0.195398,4.183853,21.0
2,4,0.054054,0.091892,0.055135,0.060541,0.022703,0.077838,0.002162,0.139459,0.009730,...,0.879795,0.375135,17.0,4.396347,2.102881,5.347357,2.420347,0.180549,5.146196,82.0
3,4,0.071806,0.080253,0.069694,0.055966,0.030623,0.080253,0.001056,0.165787,0.013728,...,1.036082,0.376980,30.0,4.421817,2.076899,5.135905,2.673849,0.178148,5.596125,86.0
4,5,0.033333,0.038889,0.033333,0.061111,0.044444,0.094444,0.000000,0.177778,0.016667,...,0.878049,0.338889,9.0,4.596490,2.064892,5.063814,2.591143,0.161879,4.702588,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,3,0.084746,0.059322,0.072034,0.067797,0.055085,0.042373,0.000000,0.165254,0.004237,...,1.067961,0.453390,10.0,4.404882,2.118607,5.127013,2.415114,0.204966,3.565189,14.0
1535,4,0.074074,0.055556,0.129630,0.074074,0.046296,0.055556,0.000000,0.111111,0.000000,...,1.000000,0.481481,3.0,4.578900,2.106413,5.049481,2.350449,0.219363,3.443641,6.0
1536,3,0.071429,0.098214,0.044643,0.044643,0.035714,0.107143,0.000000,0.214286,0.008929,...,1.061224,0.419643,4.0,4.368046,2.086000,5.140270,2.389186,0.172722,4.055240,7.0
1537,5,0.032895,0.065789,0.052632,0.072368,0.032895,0.078947,0.000000,0.138158,0.013158,...,0.830986,0.368421,3.0,4.474590,2.176371,5.095055,2.484435,0.196328,3.690323,11.0


In [ ]:
# Word Frequency (logarithm)

frequency = pd.read_csv('lexical_feature_data/SUBTLEXusExcel2007.csv')
frequency_dict_original = dict(zip(frequency['Word'], frequency['Lg10WF']))
frequency_dict = {(word.lower() if isinstance(word, str) else word): value for word, value in frequency_dict_original.items()}

for i in coherence_processed_transcripts.index:  

    string = coherence_processed_transcripts['Text'].iloc[i]
    string_split = (string.lower()).split(' ')
    frequency_values = []
    for item in string_split:
        frequency_value = frequency_dict.get(item)
        if frequency_value is not None:
            frequency_values.append(frequency_value)
    frequency_mean = np.nanmean(np.array(frequency_values))

    spacy_linguistic_features.loc[i, 'log10_freq_mean'] = frequency_mean

display(spacy_linguistic_features)

,Coherence,POS:ADJ,POS:ADP,POS:ADV,POS:AUX,POS:CCONJ,POS:DET,POS:INTJ,POS:NOUN,POS:NUM,...,POS:PUNCT,POS:SCONJ,POS:SPACE,POS:SYM,POS:VERB,POS:X,open_closed_ratio,propositional_density,n_logical_operators,log10_freq_mean
0,3,0.044534,0.064777,0.076923,0.068826,0.020243,0.093117,0.000000,0.141700,0.008097,...,0.125506,0.024291,0.0,0.0,0.125506,0.0,0.820513,0.356275,4.0,4.682792
1,4,0.043046,0.076159,0.076159,0.059603,0.026490,0.076159,0.000000,0.109272,0.009934,...,0.125828,0.016556,0.0,0.0,0.142384,0.0,0.788732,0.380795,7.0,4.666692
2,4,0.054054,0.091892,0.055135,0.060541,0.022703,0.077838,0.002162,0.139459,0.009730,...,0.152432,0.028108,0.0,0.0,0.123243,0.0,0.879795,0.375135,17.0,4.396347
3,4,0.071806,0.080253,0.069694,0.055966,0.030623,0.080253,0.001056,0.165787,0.013728,...,0.141499,0.007392,0.0,0.0,0.117212,0.0,1.036082,0.376980,30.0,4.421817
4,5,0.033333,0.038889,0.033333,0.061111,0.044444,0.094444,0.000000,0.177778,0.016667,...,0.138889,0.033333,0.0,0.0,0.155556,0.0,0.878049,0.338889,9.0,4.596490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,3,0.084746,0.059322,0.072034,0.067797,0.055085,0.042373,0.000000,0.165254,0.004237,...,0.097458,0.038136,0.0,0.0,0.144068,0.0,1.067961,0.453390,10.0,4.404882
1535,4,0.074074,0.055556,0.129630,0.074074,0.046296,0.055556,0.000000,0.111111,0.000000,...,0.092593,0.037037,0.0,0.0,0.138889,0.0,1.000000,0.481481,3.0,4.578900
1536,3,0.071429,0.098214,0.044643,0.044643,0.035714,0.107143,0.000000,0.214286,0.008929,...,0.098214,0.035714,0.0,0.0,0.133929,0.0,1.061224,0.419643,4.0,4.368046
1537,5,0.032895,0.065789,0.052632,0.072368,0.032895,0.078947,0.000000,0.138158,0.013158,...,0.118421,0.019737,0.0,0.0,0.164474,0.0,0.830986,0.368421,3.0,4.474590


In [ ]:
# Semantic Diversity

semantic_diversity = pd.read_csv('lexical_feature_data/13428_2012_278_MOESM1_ESM.csv', dtype = str)
semantic_diversity = (semantic_diversity.rename(columns = {'Supplementary Materials: SemD values': 'word', 'Unnamed: 2': 'sem_diversity'})).drop(0)
semantic_diversity_dict_original = dict(zip(semantic_diversity['word'], semantic_diversity['sem_diversity']))
semantic_diversity_dict = {(word.lower() if isinstance(word, str) else word): (float(value)) for word, value in semantic_diversity_dict_original.items()}

for i in coherence_processed_transcripts.index:  

    string = coherence_processed_transcripts['Text'].iloc[i]
    string_split = (string.lower()).split(' ')
    sem_div_values = []
    for item in string_split:
        sem_div_value = semantic_diversity_dict.get(item)
        if sem_div_value is not None:
            sem_div_values.append(sem_div_value)
    semantic_diversity_mean = np.nanmean(np.array(sem_div_values), dtype = float)

    spacy_linguistic_features.loc[i, 'semantic_diversity_mean'] = semantic_diversity_mean

display(spacy_linguistic_features)

,Coherence,POS:ADJ,POS:ADP,POS:ADV,POS:AUX,POS:CCONJ,POS:DET,POS:INTJ,POS:NOUN,POS:NUM,...,POS:SCONJ,POS:SPACE,POS:SYM,POS:VERB,POS:X,open_closed_ratio,propositional_density,n_logical_operators,log10_freq_mean,semantic_diversity_mean
0,3,0.044534,0.064777,0.076923,0.068826,0.020243,0.093117,0.000000,0.141700,0.008097,...,0.024291,0.0,0.0,0.125506,0.0,0.820513,0.356275,4.0,4.682792,2.107979
1,4,0.043046,0.076159,0.076159,0.059603,0.026490,0.076159,0.000000,0.109272,0.009934,...,0.016556,0.0,0.0,0.142384,0.0,0.788732,0.380795,7.0,4.666692,2.069585
2,4,0.054054,0.091892,0.055135,0.060541,0.022703,0.077838,0.002162,0.139459,0.009730,...,0.028108,0.0,0.0,0.123243,0.0,0.879795,0.375135,17.0,4.396347,2.102881
3,4,0.071806,0.080253,0.069694,0.055966,0.030623,0.080253,0.001056,0.165787,0.013728,...,0.007392,0.0,0.0,0.117212,0.0,1.036082,0.376980,30.0,4.421817,2.076899
4,5,0.033333,0.038889,0.033333,0.061111,0.044444,0.094444,0.000000,0.177778,0.016667,...,0.033333,0.0,0.0,0.155556,0.0,0.878049,0.338889,9.0,4.596490,2.064892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,3,0.084746,0.059322,0.072034,0.067797,0.055085,0.042373,0.000000,0.165254,0.004237,...,0.038136,0.0,0.0,0.144068,0.0,1.067961,0.453390,10.0,4.404882,2.118607
1535,4,0.074074,0.055556,0.129630,0.074074,0.046296,0.055556,0.000000,0.111111,0.000000,...,0.037037,0.0,0.0,0.138889,0.0,1.000000,0.481481,3.0,4.578900,2.106413
1536,3,0.071429,0.098214,0.044643,0.044643,0.035714,0.107143,0.000000,0.214286,0.008929,...,0.035714,0.0,0.0,0.133929,0.0,1.061224,0.419643,4.0,4.368046,2.086000
1537,5,0.032895,0.065789,0.052632,0.072368,0.032895,0.078947,0.000000,0.138158,0.013158,...,0.019737,0.0,0.0,0.164474,0.0,0.830986,0.368421,3.0,4.474590,2.176371


In [ ]:
# Age of Acquisition

aoa = pd.read_csv('lexical_feature_data/AoA_ratings_Kuperman_et_al_BRM.txt', sep = '\t')
aoa_dict_original = dict(zip(aoa['Word'], aoa['Rating.Mean']))
aoa_dict = {(word.lower() if isinstance(word, str) else word): (float(value)) for word, value in aoa_dict_original.items()}

for i in coherence_processed_transcripts.index:  

    string = coherence_processed_transcripts['Text'].iloc[i]
    string_split = (string.lower()).split(' ')
    aoa_values = []
    for item in string_split:
        aoa_value = aoa_dict.get(item)
        if aoa_value is not None:
            aoa_values.append(aoa_value)
    aoa_mean = np.nanmean(np.array(aoa_values), dtype = float)

    spacy_linguistic_features.loc[i, 'aoa_mean'] = aoa_mean

display(spacy_linguistic_features)

# TODO: make these all into a more succinct function

,Coherence,POS:ADJ,POS:ADP,POS:ADV,POS:AUX,POS:CCONJ,POS:DET,POS:INTJ,POS:NOUN,POS:NUM,...,POS:SPACE,POS:SYM,POS:VERB,POS:X,open_closed_ratio,propositional_density,n_logical_operators,log10_freq_mean,semantic_diversity_mean,aoa_mean
0,3,0.044534,0.064777,0.076923,0.068826,0.020243,0.093117,0.000000,0.141700,0.008097,...,0.0,0.0,0.125506,0.0,0.820513,0.356275,4.0,4.682792,2.107979,4.616221
1,4,0.043046,0.076159,0.076159,0.059603,0.026490,0.076159,0.000000,0.109272,0.009934,...,0.0,0.0,0.142384,0.0,0.788732,0.380795,7.0,4.666692,2.069585,4.606250
2,4,0.054054,0.091892,0.055135,0.060541,0.022703,0.077838,0.002162,0.139459,0.009730,...,0.0,0.0,0.123243,0.0,0.879795,0.375135,17.0,4.396347,2.102881,5.347357
3,4,0.071806,0.080253,0.069694,0.055966,0.030623,0.080253,0.001056,0.165787,0.013728,...,0.0,0.0,0.117212,0.0,1.036082,0.376980,30.0,4.421817,2.076899,5.135905
4,5,0.033333,0.038889,0.033333,0.061111,0.044444,0.094444,0.000000,0.177778,0.016667,...,0.0,0.0,0.155556,0.0,0.878049,0.338889,9.0,4.596490,2.064892,5.063814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,3,0.084746,0.059322,0.072034,0.067797,0.055085,0.042373,0.000000,0.165254,0.004237,...,0.0,0.0,0.144068,0.0,1.067961,0.453390,10.0,4.404882,2.118607,5.127013
1535,4,0.074074,0.055556,0.129630,0.074074,0.046296,0.055556,0.000000,0.111111,0.000000,...,0.0,0.0,0.138889,0.0,1.000000,0.481481,3.0,4.578900,2.106413,5.049481
1536,3,0.071429,0.098214,0.044643,0.044643,0.035714,0.107143,0.000000,0.214286,0.008929,...,0.0,0.0,0.133929,0.0,1.061224,0.419643,4.0,4.368046,2.086000,5.140270
1537,5,0.032895,0.065789,0.052632,0.072368,0.032895,0.078947,0.000000,0.138158,0.013158,...,0.0,0.0,0.164474,0.0,0.830986,0.368421,3.0,4.474590,2.176371,5.095055


In [ ]:
# Concreteness

concreteness = pd.read_csv('lexical_feature_data/concreteness_ratings.csv')
concreteness_dict_original = dict(zip(concreteness['Word'], concreteness['Conc.M']))
concreteness_dict = {(word.lower() if isinstance(word, str) else word): (float(value)) for word, value in concreteness_dict_original.items()}

for i in coherence_processed_transcripts.index:  

    string = coherence_processed_transcripts['Text'].iloc[i]
    string_split = (string.lower()).split(' ')
    concreteness_values = []
    for item in string_split:
        concreteness_value = concreteness_dict.get(item)
        if concreteness_value is not None:
            concreteness_values.append(concreteness_value)
    concreteness_mean = np.nanmean(np.array(concreteness_values), dtype = float)

    spacy_linguistic_features.loc[i, 'concreteness_mean'] = concreteness_mean

display(spacy_linguistic_features)

# TODO: make these all into a more succinct function

,Coherence,POS:ADJ,POS:ADP,POS:ADV,POS:AUX,POS:CCONJ,POS:DET,POS:INTJ,POS:NOUN,POS:NUM,...,POS:SYM,POS:VERB,POS:X,open_closed_ratio,propositional_density,n_logical_operators,log10_freq_mean,semantic_diversity_mean,aoa_mean,concreteness_mean
0,3,0.044534,0.064777,0.076923,0.068826,0.020243,0.093117,0.000000,0.141700,0.008097,...,0.0,0.125506,0.0,0.820513,0.356275,4.0,4.682792,2.107979,4.616221,2.597958
1,4,0.043046,0.076159,0.076159,0.059603,0.026490,0.076159,0.000000,0.109272,0.009934,...,0.0,0.142384,0.0,0.788732,0.380795,7.0,4.666692,2.069585,4.606250,2.615833
2,4,0.054054,0.091892,0.055135,0.060541,0.022703,0.077838,0.002162,0.139459,0.009730,...,0.0,0.123243,0.0,0.879795,0.375135,17.0,4.396347,2.102881,5.347357,2.420347
3,4,0.071806,0.080253,0.069694,0.055966,0.030623,0.080253,0.001056,0.165787,0.013728,...,0.0,0.117212,0.0,1.036082,0.376980,30.0,4.421817,2.076899,5.135905,2.673849
4,5,0.033333,0.038889,0.033333,0.061111,0.044444,0.094444,0.000000,0.177778,0.016667,...,0.0,0.155556,0.0,0.878049,0.338889,9.0,4.596490,2.064892,5.063814,2.591143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,3,0.084746,0.059322,0.072034,0.067797,0.055085,0.042373,0.000000,0.165254,0.004237,...,0.0,0.144068,0.0,1.067961,0.453390,10.0,4.404882,2.118607,5.127013,2.415114
1535,4,0.074074,0.055556,0.129630,0.074074,0.046296,0.055556,0.000000,0.111111,0.000000,...,0.0,0.138889,0.0,1.000000,0.481481,3.0,4.578900,2.106413,5.049481,2.350449
1536,3,0.071429,0.098214,0.044643,0.044643,0.035714,0.107143,0.000000,0.214286,0.008929,...,0.0,0.133929,0.0,1.061224,0.419643,4.0,4.368046,2.086000,5.140270,2.389186
1537,5,0.032895,0.065789,0.052632,0.072368,0.032895,0.078947,0.000000,0.138158,0.013158,...,0.0,0.164474,0.0,0.830986,0.368421,3.0,4.474590,2.176371,5.095055,2.484435


In [ ]:
# Semantic thematic distance
path = api.load("word2vec-google-news-300", return_path=True)
model = KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
# TODO: move this!

spacy_processed_transcripts = spacy_transcripts.copy()
for i in spacy_transcripts.index:
    story_string = spacy_transcripts['Text'].iloc[i]
    story_string = re.sub(r'[^A-Za-z0-9\'\s]', '', story_string)
    spacy_processed_transcripts.loc[i,'Text'] = story_string

display(spacy_processed_transcripts.loc[100, 'Text'])

'Those who are worshipped are more certain that they are doomed than their creators And when they gain their approval they simply cease to exist and will find themselves walking the same boring existence again It was good I thought I had lived so long so much and so much of this facility filled with dark demons and living hell I couldnt even begin to fathom Death should have been the only thing I had heard of the only one that had soothed me I remembered my childhood We were little children There had been kids at home there was room for some action there was fun The forest had kept us warm so that we could grow old We had played and had grown and got old and had grown older and found other family members It had been a good life but the peace was the only warmth I had I knew there were others like me somewhere lurking in the darkness I knew there were other words that could describe our existence The moonlight danced on its graceful existence and danced on my face I put my arms around m

In [ ]:
# Semantic thematic distance 
# TODO: be consistent in capitalization/stylistic choices

for i in spacy_processed_transcripts.index:  

    string = spacy_processed_transcripts['Text'].iloc[i]
    string_split = (string.lower()).split(' ')
        
    sem_sim = []
    for j in range(len(string_split) - 1):
        try:
            sem_sim_value = model.similarity(string_split[j], string_split[j + 1])
        except KeyError:
            continue
        
        sem_sim.append(float(sem_sim_value))
    
    sem_thematic_distance = np.nanmean(np.array(sem_sim))

    spacy_linguistic_features.loc[i, 'semantic_thematic_distance'] = sem_thematic_distance

display(spacy_linguistic_features)

,Coherence,POS:ADJ,POS:ADP,POS:ADV,POS:AUX,POS:CCONJ,POS:DET,POS:INTJ,POS:NOUN,POS:NUM,...,POS:VERB,POS:X,open_closed_ratio,propositional_density,n_logical_operators,log10_freq_mean,semantic_diversity_mean,aoa_mean,concreteness_mean,semantic_thematic_distance
0,3,0.044534,0.064777,0.076923,0.068826,0.020243,0.093117,0.000000,0.141700,0.008097,...,0.125506,0.0,0.820513,0.356275,4.0,4.682792,2.107979,4.616221,2.597958,0.198185
1,4,0.043046,0.076159,0.076159,0.059603,0.026490,0.076159,0.000000,0.109272,0.009934,...,0.142384,0.0,0.788732,0.380795,7.0,4.666692,2.069585,4.606250,2.615833,0.195398
2,4,0.054054,0.091892,0.055135,0.060541,0.022703,0.077838,0.002162,0.139459,0.009730,...,0.123243,0.0,0.879795,0.375135,17.0,4.396347,2.102881,5.347357,2.420347,0.180549
3,4,0.071806,0.080253,0.069694,0.055966,0.030623,0.080253,0.001056,0.165787,0.013728,...,0.117212,0.0,1.036082,0.376980,30.0,4.421817,2.076899,5.135905,2.673849,0.178148
4,5,0.033333,0.038889,0.033333,0.061111,0.044444,0.094444,0.000000,0.177778,0.016667,...,0.155556,0.0,0.878049,0.338889,9.0,4.596490,2.064892,5.063814,2.591143,0.161879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,3,0.084746,0.059322,0.072034,0.067797,0.055085,0.042373,0.000000,0.165254,0.004237,...,0.144068,0.0,1.067961,0.453390,10.0,4.404882,2.118607,5.127013,2.415114,0.204966
1535,4,0.074074,0.055556,0.129630,0.074074,0.046296,0.055556,0.000000,0.111111,0.000000,...,0.138889,0.0,1.000000,0.481481,3.0,4.578900,2.106413,5.049481,2.350449,0.219363
1536,3,0.071429,0.098214,0.044643,0.044643,0.035714,0.107143,0.000000,0.214286,0.008929,...,0.133929,0.0,1.061224,0.419643,4.0,4.368046,2.086000,5.140270,2.389186,0.172722
1537,5,0.032895,0.065789,0.052632,0.072368,0.032895,0.078947,0.000000,0.138158,0.013158,...,0.164474,0.0,0.830986,0.368421,3.0,4.474590,2.176371,5.095055,2.484435,0.196328


In [ ]:
# Surprisal

model = scorer.IncrementalLMScorer('gpt2')

for i in range(len(story_lists)):
    story_list = story_lists[i]
    transcript_surprisal = []
    for j in range(len(story_list)):
        sentence = story_list[j]
        sentence_surprisal = model.sequence_score(sentence, reduction = lambda x: -x.mean(0).item())
        transcript_surprisal.append(sentence_surprisal)
    transcript_surprisal_array = np.array(transcript_surprisal)
    surprisal_mean = np.nanmean(transcript_surprisal_array)

    spacy_linguistic_features.loc[i, 'surprisal'] = surprisal_mean

display(spacy_linguistic_features)

,Coherence,POS:ADJ,POS:ADP,POS:ADV,POS:AUX,POS:CCONJ,POS:DET,POS:INTJ,POS:NOUN,POS:NUM,...,POS:X,open_closed_ratio,propositional_density,n_logical_operators,log10_freq_mean,semantic_diversity_mean,aoa_mean,concreteness_mean,semantic_thematic_distance,surprisal
0,3,0.044534,0.064777,0.076923,0.068826,0.020243,0.093117,0.000000,0.141700,0.008097,...,0.0,0.820513,0.356275,4.0,4.682792,2.107979,4.616221,2.597958,0.198185,4.895232
1,4,0.043046,0.076159,0.076159,0.059603,0.026490,0.076159,0.000000,0.109272,0.009934,...,0.0,0.788732,0.380795,7.0,4.666692,2.069585,4.606250,2.615833,0.195398,4.183853
2,4,0.054054,0.091892,0.055135,0.060541,0.022703,0.077838,0.002162,0.139459,0.009730,...,0.0,0.879795,0.375135,17.0,4.396347,2.102881,5.347357,2.420347,0.180549,5.146196
3,4,0.071806,0.080253,0.069694,0.055966,0.030623,0.080253,0.001056,0.165787,0.013728,...,0.0,1.036082,0.376980,30.0,4.421817,2.076899,5.135905,2.673849,0.178148,5.596125
4,5,0.033333,0.038889,0.033333,0.061111,0.044444,0.094444,0.000000,0.177778,0.016667,...,0.0,0.878049,0.338889,9.0,4.596490,2.064892,5.063814,2.591143,0.161879,4.702588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,3,0.084746,0.059322,0.072034,0.067797,0.055085,0.042373,0.000000,0.165254,0.004237,...,0.0,1.067961,0.453390,10.0,4.404882,2.118607,5.127013,2.415114,0.204966,3.565189
1535,4,0.074074,0.055556,0.129630,0.074074,0.046296,0.055556,0.000000,0.111111,0.000000,...,0.0,1.000000,0.481481,3.0,4.578900,2.106413,5.049481,2.350449,0.219363,3.443641
1536,3,0.071429,0.098214,0.044643,0.044643,0.035714,0.107143,0.000000,0.214286,0.008929,...,0.0,1.061224,0.419643,4.0,4.368046,2.086000,5.140270,2.389186,0.172722,4.055240
1537,5,0.032895,0.065789,0.052632,0.072368,0.032895,0.078947,0.000000,0.138158,0.013158,...,0.0,0.830986,0.368421,3.0,4.474590,2.176371,5.095055,2.484435,0.196328,3.690323


In [ ]:
spacy_linguistic_features

,Coherence,POS:ADJ,POS:ADP,POS:ADV,POS:AUX,POS:CCONJ,POS:DET,POS:INTJ,POS:NOUN,POS:NUM,...,POS:X,open_closed_ratio,propositional_density,n_logical_operators,log10_freq_mean,semantic_diversity_mean,aoa_mean,concreteness_mean,semantic_thematic_distance,surprisal
0,3,0.044534,0.064777,0.076923,0.068826,0.020243,0.093117,0.000000,0.141700,0.008097,...,0.0,0.820513,0.356275,4.0,4.682792,2.107979,4.616221,2.597958,0.198185,4.895232
1,4,0.043046,0.076159,0.076159,0.059603,0.026490,0.076159,0.000000,0.109272,0.009934,...,0.0,0.788732,0.380795,7.0,4.666692,2.069585,4.606250,2.615833,0.195398,4.183853
2,4,0.054054,0.091892,0.055135,0.060541,0.022703,0.077838,0.002162,0.139459,0.009730,...,0.0,0.879795,0.375135,17.0,4.396347,2.102881,5.347357,2.420347,0.180549,5.146196
3,4,0.071806,0.080253,0.069694,0.055966,0.030623,0.080253,0.001056,0.165787,0.013728,...,0.0,1.036082,0.376980,30.0,4.421817,2.076899,5.135905,2.673849,0.178148,5.596125
4,5,0.033333,0.038889,0.033333,0.061111,0.044444,0.094444,0.000000,0.177778,0.016667,...,0.0,0.878049,0.338889,9.0,4.596490,2.064892,5.063814,2.591143,0.161879,4.702588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,3,0.084746,0.059322,0.072034,0.067797,0.055085,0.042373,0.000000,0.165254,0.004237,...,0.0,1.067961,0.453390,10.0,4.404882,2.118607,5.127013,2.415114,0.204966,3.565189
1535,4,0.074074,0.055556,0.129630,0.074074,0.046296,0.055556,0.000000,0.111111,0.000000,...,0.0,1.000000,0.481481,3.0,4.578900,2.106413,5.049481,2.350449,0.219363,3.443641
1536,3,0.071429,0.098214,0.044643,0.044643,0.035714,0.107143,0.000000,0.214286,0.008929,...,0.0,1.061224,0.419643,4.0,4.368046,2.086000,5.140270,2.389186,0.172722,4.055240
1537,5,0.032895,0.065789,0.052632,0.072368,0.032895,0.078947,0.000000,0.138158,0.013158,...,0.0,0.830986,0.368421,3.0,4.474590,2.176371,5.095055,2.484435,0.196328,3.690323


In [ ]:
spacy_linguistic_features.to_csv('linguistic_features/spacy_linguistic_features.csv')